In [29]:
import os
import pandas as pd
import matplotlib.pylab as plt
# data cleaning and manipulation 
import numpy as np
import random


data = pd.read_csv('clinical.csv')

In [30]:
data.head()

,PATIENT_NAME,CANCER_TYPE_ACRONYM,AGE,DAYS_TO_BIRTH,PERSON_NEOPLASM_CANCER_STATUS,RACE,TISSUE_SOURCE_SITE,ANEUPLOIDY_SCORE,GRADE,IGF2,COL1A1,EEF1A1,CLU,H19,GAPDH,CD74,C3,RPL8,RPS18
0,TCGA-04-1348,1,44,16236,1,0,4,5,2,0.009529,0.069028,0.183659,0.028697,0.197605,0.449789,1.000000,0.131190,0.145715,0.165588
1,TCGA-04-1357,1,52,18999,1,0,4,12,2,0.040637,0.144819,0.658017,0.501043,0.004170,0.359466,1.000000,0.126835,0.359117,0.336082
2,TCGA-04-1362,1,59,21745,1,0,4,16,2,0.493489,0.050296,0.571312,0.086541,0.000640,0.301870,0.125584,0.068805,0.368181,0.304159
3,TCGA-04-1364,1,61,22294,1,0,4,31,2,1.000000,0.011310,0.164350,0.012596,0.009681,0.050258,0.004711,0.012590,0.149953,0.119808
4,TCGA-04-1365,1,87,31925,1,0,4,9,2,0.019999,0.052526,0.334952,0.105748,1.000000,0.483308,0.421103,0.073854,0.298796,0.221242


In [68]:
X = data.iloc[:,6:10]
y= data.iloc[:,4]
y.head()

0    1
1    1
2    1
3    1
4    1
Name: PERSON_NEOPLASM_CANCER_STATUS, dtype: int64

In [33]:
#Splitting Dataset
#Now let's split our data into training and testing datasets.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [34]:
#Scaling ensures that just because some features are big, the model won't lead to using them as a main predictor

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


# 2. Preparing ANN
Importing Keras and initialising ANN
Building the layers
Let's build the Layers. 
We can play around and change number of units but if we are not sure what number to initialize with then simply 
initialize the units of all layers except the last one with the (number of features + number of output nodes)/2 which
equals to 3 in our case. My results were improved by setting units = 3 for the first layer and decreasing the units
in the hidden layers. Also we have to provide input dimension for the first layer only. *relu' reffers to rectified
linear unit and sigmoid reffers to sigmoid activation function*. With the help of sigmoid activation function, we can
get the probabilities of the classification which might be benificial in some cases to conduct further research.

Tuning Hyper parameters
Let's first find the hyper parameters using which model can give more accurate predictions. Here I'll tune batch_size,
epochs and optimizer.


In [69]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))
    classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [24, 48],
               'epochs': [56, 112],
               'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 10)
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/56
200/200 [==============================] - 15s 77ms/step - loss: 0.6921 - acc: 0.7600
Epoch 2/56
200/200 [==============================] - 0s 392us/step - loss: 0.6898 - acc: 0.7800
Epoch 3/56
200/200 [==============================] - 0s 425us/step - loss: 0.6874 - acc: 0.7800
Epoch 4/56
200/200 [==============================] - 0s 399us/step - loss: 0.6849 - acc: 0.7800
Epoch 5/56
200/200 [==============================] - 0s 372us/step - loss: 0.6826 - acc: 0.7800
Epoch 6/56
200/200 [==============================] - 0s 456us/step - loss: 0.6803 - acc: 0.7800
Epoch 7/56
200/200 [==============================] - 0s 403us/step - loss: 0.6781 - acc: 0.7800
Epoch 8/56
200/200 [==============================] - 0s 430us/step - loss: 0.6758 - acc: 0.7800
Epoch 9/56
200/200 [==============================] - 0s 494us/step - loss: 0.6735 - acc: 0.7800
Epoch 10/56
200/200 [==============================] - 0s 427us/step - loss: 0.6713 - acc: 0.7800
Epoch 11/56
200/200 [========

200/200 [==============================] - 0s 587us/step - loss: 0.6063 - acc: 0.7600
Epoch 1/56
201/201 [==============================] - 10s 50ms/step - loss: 0.6921 - acc: 0.7015
Epoch 2/56
201/201 [==============================] - 0s 417us/step - loss: 0.6899 - acc: 0.7711
Epoch 3/56
201/201 [==============================] - 0s 470us/step - loss: 0.6875 - acc: 0.7711
Epoch 4/56
201/201 [==============================] - 0s 471us/step - loss: 0.6853 - acc: 0.7711
Epoch 5/56
201/201 [==============================] - 0s 408us/step - loss: 0.6830 - acc: 0.7711
Epoch 6/56
201/201 [==============================] - 0s 361us/step - loss: 0.6808 - acc: 0.7711
Epoch 7/56
201/201 [==============================] - 0s 400us/step - loss: 0.6787 - acc: 0.7711
Epoch 8/56
201/201 [==============================] - 0s 405us/step - loss: 0.6765 - acc: 0.7711
Epoch 9/56
201/201 [==============================] - 0s 416us/step - loss: 0.6745 - acc: 0.7711
Epoch 10/56
201/201 [====================

Epoch 28/56
201/201 [==============================] - 0s 375us/step - loss: 0.6364 - acc: 0.7811
Epoch 29/56
201/201 [==============================] - 0s 381us/step - loss: 0.6345 - acc: 0.7811
Epoch 30/56
201/201 [==============================] - 0s 385us/step - loss: 0.6328 - acc: 0.7811
Epoch 31/56
201/201 [==============================] - 0s 383us/step - loss: 0.6312 - acc: 0.7811
Epoch 32/56
201/201 [==============================] - 0s 398us/step - loss: 0.6296 - acc: 0.7811
Epoch 33/56
201/201 [==============================] - 0s 376us/step - loss: 0.6280 - acc: 0.7811
Epoch 34/56
201/201 [==============================] - 0s 387us/step - loss: 0.6263 - acc: 0.7811
Epoch 35/56
201/201 [==============================] - 0s 373us/step - loss: 0.6248 - acc: 0.7811
Epoch 36/56
201/201 [==============================] - 0s 390us/step - loss: 0.6232 - acc: 0.7811
Epoch 37/56
201/201 [==============================] - 0s 441us/step - loss: 0.6216 - acc: 0.7811
Epoch 38/56
201/201 

201/201 [==============================] - 0s 420us/step - loss: 0.5971 - acc: 0.7861
Epoch 27/56
201/201 [==============================] - 0s 403us/step - loss: 0.5916 - acc: 0.7861
Epoch 28/56
201/201 [==============================] - 0s 407us/step - loss: 0.5863 - acc: 0.7861
Epoch 29/56
201/201 [==============================] - 0s 404us/step - loss: 0.5807 - acc: 0.7861
Epoch 30/56
201/201 [==============================] - 0s 435us/step - loss: 0.5744 - acc: 0.7861
Epoch 31/56
201/201 [==============================] - 0s 420us/step - loss: 0.5691 - acc: 0.7861
Epoch 32/56
201/201 [==============================] - 0s 416us/step - loss: 0.5634 - acc: 0.7861
Epoch 33/56
201/201 [==============================] - 0s 449us/step - loss: 0.5586 - acc: 0.7861
Epoch 34/56
201/201 [==============================] - 0s 480us/step - loss: 0.5535 - acc: 0.7861
Epoch 35/56
201/201 [==============================] - 0s 446us/step - loss: 0.5490 - acc: 0.7861
Epoch 36/56
201/201 [===========

200/200 [==============================] - 0s 398us/step - loss: 0.6409 - acc: 0.7800
Epoch 25/56
200/200 [==============================] - 0s 400us/step - loss: 0.6389 - acc: 0.7800
Epoch 26/56
200/200 [==============================] - 0s 377us/step - loss: 0.6371 - acc: 0.7800
Epoch 27/56
200/200 [==============================] - 0s 390us/step - loss: 0.6352 - acc: 0.7800
Epoch 28/56
200/200 [==============================] - 0s 383us/step - loss: 0.6335 - acc: 0.7800
Epoch 29/56
200/200 [==============================] - 0s 491us/step - loss: 0.6316 - acc: 0.7800
Epoch 30/56
200/200 [==============================] - 0s 526us/step - loss: 0.6298 - acc: 0.7800
Epoch 31/56
200/200 [==============================] - 0s 925us/step - loss: 0.6281 - acc: 0.7800
Epoch 32/56
200/200 [==============================] - 0s 416us/step - loss: 0.6263 - acc: 0.7800
Epoch 33/56
200/200 [==============================] - 0s 391us/step - loss: 0.6245 - acc: 0.7800
Epoch 34/56
200/200 [===========

...

201/201 [==============================] - 0s 306us/step - loss: 0.5759 - acc: 0.7662
Epoch 77/112
201/201 [==============================] - 0s 307us/step - loss: 0.5750 - acc: 0.7662
Epoch 78/112
201/201 [==============================] - 0s 309us/step - loss: 0.5738 - acc: 0.7662
Epoch 79/112
201/201 [==============================] - 0s 306us/step - loss: 0.5723 - acc: 0.7662
Epoch 80/112
201/201 [==============================] - 0s 315us/step - loss: 0.5707 - acc: 0.7662
Epoch 81/112
201/201 [==============================] - 0s 299us/step - loss: 0.5694 - acc: 0.7662
Epoch 82/112
201/201 [==============================] - 0s 298us/step - loss: 0.5683 - acc: 0.7662
Epoch 83/112
201/201 [==============================] - 0s 338us/step - loss: 0.5672 - acc: 0.7662
Epoch 84/112
201/201 [==============================] - 0s 302us/step - loss: 0.5666 - acc: 0.7662
Epoch 85/112
201/201 [==============================] - 0s 288us/step - loss: 0.5656 - acc: 0.7662
Epoch 86/112
201/201 [=

201/201 [==============================] - 0s 284us/step - loss: 0.6704 - acc: 0.7861
Epoch 17/112
201/201 [==============================] - 0s 375us/step - loss: 0.6693 - acc: 0.7861
Epoch 18/112
201/201 [==============================] - 0s 404us/step - loss: 0.6680 - acc: 0.7861
Epoch 19/112
201/201 [==============================] - 0s 351us/step - loss: 0.6666 - acc: 0.7861
Epoch 20/112
201/201 [==============================] - 0s 365us/step - loss: 0.6654 - acc: 0.7861
Epoch 21/112
201/201 [==============================] - 0s 280us/step - loss: 0.6640 - acc: 0.7861
Epoch 22/112
201/201 [==============================] - 0s 286us/step - loss: 0.6628 - acc: 0.7861
Epoch 23/112
201/201 [==============================] - 0s 281us/step - loss: 0.6616 - acc: 0.7861
Epoch 24/112
201/201 [==============================] - 0s 286us/step - loss: 0.6606 - acc: 0.7861
Epoch 25/112
201/201 [==============================] - 0s 269us/step - loss: 0.6593 - acc: 0.7861
Epoch 26/112
201/201 [=

201/201 [==============================] - 0s 283us/step - loss: 0.6144 - acc: 0.7811
Epoch 69/112
201/201 [==============================] - 0s 273us/step - loss: 0.6134 - acc: 0.7811
Epoch 70/112
201/201 [==============================] - 0s 284us/step - loss: 0.6126 - acc: 0.7811
Epoch 71/112
201/201 [==============================] - 0s 288us/step - loss: 0.6116 - acc: 0.7811
Epoch 72/112
201/201 [==============================] - 0s 277us/step - loss: 0.6107 - acc: 0.7811
Epoch 73/112
201/201 [==============================] - 0s 284us/step - loss: 0.6097 - acc: 0.7811
Epoch 74/112
201/201 [==============================] - 0s 272us/step - loss: 0.6089 - acc: 0.7811
Epoch 75/112
201/201 [==============================] - 0s 287us/step - loss: 0.6080 - acc: 0.7811
Epoch 76/112
201/201 [==============================] - 0s 277us/step - loss: 0.6071 - acc: 0.7811
Epoch 77/112
201/201 [==============================] - 0s 280us/step - loss: 0.6062 - acc: 0.7811
Epoch 78/112
201/201 [=

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/56
223/223 [==============================] - 15s 66ms/step - loss: 0.6921 - acc: 0.7309
Epoch 2/56
223/223 [==============================] - 0s 551us/step - loss: 0.6895 - acc: 0.7758
Epoch 3/56
223/223 [==============================] - 0s 541us/step - loss: 0.6868 - acc: 0.7758
Epoch 4/56
223/223 [==============================] - 0s 557us/step - loss: 0.6841 - acc: 0.7758
Epoch 5/56
223/223 [==============================] - 0s 541us/step - loss: 0.6817 - acc: 0.7758
Epoch 6/56
223/223 [==============================] - 0s 553us/step - loss: 0.6791 - acc: 0.7758
Epoch 7/56
223/223 [==============================] - 0s 531us/step - loss: 0.6767 - acc: 0.7758
Epoch 8/56
223/223 [==============================] - 0s 534us/step - loss: 0.6744 - acc: 0.7758
Epoch 9/56
223/223 [==============================] - 0s 556us/step - loss: 0.6719 - acc: 0.7758
Epoch 10/56
223/223 [==============================] - 0s 537us/step - loss: 0.6697 - acc: 0.7758
Epoch 11/56
223/223 [========

In [36]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("best_parameters: ")
print(best_parameters)
print("\nbest_accuracy: ")
print(best_accuracy)

best_parameters: 
{'batch_size': 24, 'epochs': 56, 'optimizer': 'adam'}

best_accuracy: 
0.7757847533632287


In [37]:
#building the ANN
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))
classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [38]:
#Compiling classifier. Using adam optimizer. Using binary_crossentropy for loss function since classification is 
#binary, i.e. only two classes '0' or '1'
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [39]:
#Fitting the data
classifier.fit(X_train, y_train, batch_size = 24, epochs = 56)

Epoch 1/56
223/223 [==============================] - 11s 49ms/step - loss: 0.6908 - acc: 0.7399
Epoch 2/56
223/223 [==============================] - 0s 348us/step - loss: 0.6871 - acc: 0.7758
Epoch 3/56
223/223 [==============================] - 0s 352us/step - loss: 0.6841 - acc: 0.7758
Epoch 4/56
223/223 [==============================] - 0s 354us/step - loss: 0.6813 - acc: 0.7758
Epoch 5/56
223/223 [==============================] - 0s 335us/step - loss: 0.6783 - acc: 0.7758
Epoch 6/56
223/223 [==============================] - 0s 347us/step - loss: 0.6752 - acc: 0.7758
Epoch 7/56
223/223 [==============================] - 0s 344us/step - loss: 0.6724 - acc: 0.7758
Epoch 8/56
223/223 [==============================] - 0s 352us/step - loss: 0.6695 - acc: 0.7758
Epoch 9/56
223/223 [==============================] - 0s 341us/step - loss: 0.6664 - acc: 0.7758
Epoch 10/56
223/223 [==============================] - 0s 355us/step - loss: 0.6633 - acc: 0.7758
Epoch 11/56
223/223 [========

In [61]:
#3Making Predictions
#Now y_pred contains the probability of a person having cancer. We'll assign the results true or
#false based on their probabilities (if probability >= 0.5 than true else false)

from keras.models import load_model

classifier.save('ovarian_cancer') #Save trained ANN
#classifier = load_model('ovarian_cancer')  #Load trained ANN

In [66]:
y_pred = classifier.predict(X_test)
y_pred = [ 1 if y>=0.5 else 0 for y in y_pred ]

In [67]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
print("Accuracy: "+ str(accuracy*100)+"%")

[[ 0 12]
 [ 0 44]]
Accuracy: 78.57142857142857%
